### Joslin

### Setup Environment:

In [4]:
import os
import pandas as pd

from src.classifiers import process_labels, split_data
from src.classifiers_base import preprocess_df

from transformers import BertTokenizer

from src.multimodal_data_loader import VQADataset
from torch.utils.data import DataLoader
import pandas as pd
from src.classifiers_base import train_early_fusion, train_late_fusion

In [5]:
PATH = 'datasets/joslin/'

In [6]:
text_path = os.path.join(PATH, 'labels.csv')
images_path = os.path.join(PATH, 'images')

## Get data

In [7]:
df = pd.read_csv(text_path)
df

,ID,EYE_DR,text,DR_2,DR_3,split,img_path,img_exist
0,67870,2,An image from the left eye of a female patien...,1,1,train,/media/enc/vera1/sebastian/codes/Foundational-...,True
1,19446,2,An image from the left eye of a female patien...,1,1,train,/media/enc/vera1/sebastian/codes/Foundational-...,True
2,113968,2,An image from the right eye of a male patient...,1,1,train,/media/enc/vera1/sebastian/codes/Foundational-...,True
3,50528,1,An image from the right eye of a male patient...,1,1,train,/media/enc/vera1/sebastian/codes/Foundational-...,True
4,91254,0,An image from the right eye of a male American...,0,0,train,/media/enc/vera1/sebastian/codes/Foundational-...,True
...,...,...,...,...,...,...,...,...
23515,148958,1,"An image from the left eye of a male patient,...",1,1,train,/media/enc/vera1/sebastian/codes/Foundational-...,True
23516,96198,0,An image from the right eye of a no sex report...,0,0,train,/media/enc/vera1/sebastian/codes/Foundational-...,True
23517,82724,2,An image from the left eye of a female White p...,1,1,train,/media/enc/vera1/sebastian/codes/Foundational-...,True
23518,173196,1,An image from the left eye of a female patien...,1,1,train,/media/enc/vera1/sebastian/codes/Foundational-...,True


In [8]:
df.columns

Index(['ID', 'EYE_DR', 'text', 'DR_2', 'DR_3', 'split', 'img_path',
       'img_exist'],
      dtype='object')

## Data Perparation

In [11]:
# Select features and labels vectors
text_columns = 'text'
image_columns = 'ID'
label_columns = 'DR_3'

df = preprocess_df(df, image_columns, images_path)


0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [12]:
df.head()

""


In [13]:
# Split the data
train_df, test_df = split_data(df)

# Process and one-hot encode labels for training set
train_labels, mlb, train_columns = process_labels(train_df, col=label_columns)
test_labels = process_labels(test_df, col=label_columns, train_columns=train_columns)

KeyError: 'split'

In [ ]:
df

In [ ]:
# Instantiate tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
train_dataset = VQADataset(train_df, text_columns, image_columns, label_columns, mlb, train_columns, tokenizer)
test_dataset = VQADataset(test_df, text_columns, image_columns, label_columns, mlb, train_columns, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

### Models

In [ ]:
if label_columns == 'DR_2':
    output_size = 1
else:
    output_size = len(pd.unique(train_df[label_columns]))
multilabel = False

In [14]:
# Train early fusion model
print("Training Early Fusion Model:")
device = 'cuda'
train_early_fusion(train_loader, test_loader, output_size, num_epochs=30, multilabel=multilabel, report=True, lr=0.001, freeze_backbone=True, device=device)

Training Early Fusion Model:


NameError: name 'train_loader' is not defined

In [15]:
# Train late fusion model
print("Training Late Fusion Model:")
device = "cuda"
train_late_fusion(train_loader, test_loader, output_size, num_epochs=30, multilabel=multilabel, report=True, freeze_backbone=True, device = "cuda")

Training Late Fusion Model:


NameError: name 'train_loader' is not defined

invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invalid path!
invali